In [1]:
import itertools
import pandas as pd
import numpy as np
import toytree
import toyplot
import arviz as az
import pymc3 as pm

### Tree and dataframe setup

In [3]:
# generate a random tree
NSPECIES = 80
TREE = toytree.rtree.bdtree(
    ntips=NSPECIES,
    seed=666,
).mod.node_scale_root_height(1.0)

# node idxs that delimit several distinct clades on this tree
CLADES = [152, 153, 154, 155]

# draw and color the four major clades
TREE.draw(
    layout='d', 
    width=500,
    tip_labels=False,
    edge_colors=TREE.get_edge_values_mapped({
        j: toytree.colors[i] for i, j in enumerate(CLADES)
    }),
    scalebar=True,
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="ta5b73cfe168749a4986381645ccae305" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.0 0.5 1.0

In [4]:
# make group index (gidx)
crown_dict = {i: TREE.get_tip_labels(i) for i in CLADES}
gidx = np.zeros(TREE.ntips, dtype=int)
for tidx, tip in enumerate(TREE.get_tip_labels()):
    for cidx, clade in enumerate(crown_dict):
        if tip in crown_dict[clade]:
            gidx[tidx] = cidx
gidx

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [16]:
# True param values
𝛼_mean = 0.05
𝛼_std = 0.02
𝛽_mean = 3.0
𝛽_std = 0.2
𝜓_mean = 0.0
𝜓_std = 0.33

# 4 different clade effects on rate of RI (used for partial-pooling data)
𝜓_0_mean = 1.0
𝜓_0_std = 0.1
𝜓_1_mean = 0.5
𝜓_1_std = 0.05
𝜓_2_mean = -0.5
𝜓_2_std = 0.05
𝜓_3_mean = -1.0
𝜓_3_std = 0.1

In [121]:
# species dataframe
SPECIES_DATA = pd.DataFrame({
    "gidx": gidx,
    "b": np.random.normal(𝛽_mean, 𝛽_std, TREE.ntips),
    "psi": np.random.normal(𝜓_mean, 𝜓_std, TREE.ntips),
    "psi_x": np.concatenate([
        np.random.normal(𝜓_0_mean, 𝜓_0_std, len(gidx[gidx == 0])),
        np.random.normal(𝜓_1_mean, 𝜓_1_std, len(gidx[gidx == 1])),
        np.random.normal(𝜓_2_mean, 𝜓_2_std, len(gidx[gidx == 2])),
        np.random.normal(𝜓_3_mean, 𝜓_3_std, len(gidx[gidx == 3])),
    ]),
})
SPECIES_DATA.head()

,gidx,b,psi,psi_x
0,0,2.859146,-0.121095,1.125654
1,0,2.910609,-0.034003,1.111818
2,0,2.984292,-0.194640,1.021335
3,0,3.139326,-0.164365,0.851735
4,0,2.877360,0.013215,0.965630


### Generate crossing data

In [230]:
def get_dist(tree, idx0, idx1):
    "returns the genetic distance between two nodes on a tree"
    dist = tree.treenode.get_distance(
        tree.idx_dict[idx0], 
        tree.idx_dict[idx1],
    )
    return dist

# get all combinations of two sampled taxa
a, b = zip(*itertools.combinations(range(NSPECIES), 2))

# organize into DF and get genetic distance between pairs
DATA = pd.DataFrame({
    "sidx0": a,
    "sidx1": b,
    "dist": [get_dist(TREE, i, j) / 2 for (i, j) in zip(a, b)],
})

# get pairwise velocity
DATA['b'] = (
    np.random.normal(𝛽_mean, 𝛽_std, DATA.shape[0])
)

DATA['velo'] = (
    DATA['b']
    + SPECIES_DATA['psi'][DATA.sidx0].values
    + SPECIES_DATA['psi'][DATA.sidx1].values
)

DATA['velo_x'] = (
    DATA['b']
    + SPECIES_DATA['psi_x'][DATA.sidx0].values
    + SPECIES_DATA['psi_x'][DATA.sidx1].values
)

DATA['intercept'] = np.random.normal(𝛼_mean, 𝛼_std, DATA.shape[0])

# get logits
DATA['logit_b'] = (
    1 / (1 + np.exp(-(DATA.intercept + DATA.dist * DATA.b)))
)
DATA['logit'] = (
    1 / (1 + np.exp(-(DATA.intercept + DATA.dist * DATA.velo)))
)
DATA['logit_x'] = (
    1 / (1 + np.exp(-(DATA.intercept + DATA.dist * DATA.velo_x)))
)

# get RI estimates
DATA['RI_pooled'] = np.random.binomial(n=1, p=DATA.logit_b / DATA.logit_b.max())
DATA['RI_unpooled'] = np.random.binomial(n=1, p=DATA.logit / DATA.logit.max())
DATA['RI_partpooled'] = np.random.binomial(n=1, p=DATA.logit_x / DATA.logit_x.max())

DATA.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,logit_b,logit,logit_x,RI_pooled,RI_unpooled,RI_partpooled
0,0,1,0.073376,2.897884,2.742786,5.135356,0.045799,0.564252,0.561451,0.604108,0,1,1
1,0,2,0.089748,3.114409,2.798674,5.261398,0.009106,0.571658,0.564706,0.618056,1,0,1
2,0,3,0.129770,2.921486,2.636027,4.898876,0.022516,0.599081,0.590152,0.658866,1,0,0
3,0,4,0.144542,2.728969,2.621089,4.820252,0.048886,0.609054,0.605335,0.678223,1,1,1
4,0,5,0.144542,3.072658,2.791630,5.096326,0.046508,0.620257,0.610643,0.686357,1,1,1


In [131]:
NSAMPLES = 1000
SAMPLE = DATA.sample(NSAMPLES).copy().reset_index(drop=True)
SAMPLE.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,logit_b,logit,logit_x,RI_pooled,RI_unpooled,RI_partpooled
0,10,76,1.000000,3.038481,2.979717,2.885050,0.074551,0.957427,0.954966,0.950715,1,1,1
1,56,77,0.592720,2.928854,2.135937,1.590023,0.030588,0.854032,0.785262,0.725722,1,1,1
2,22,31,0.438896,3.141626,2.960778,5.087621,0.065631,0.809147,0.796588,0.908758,1,0,1
3,41,74,1.000000,3.041354,2.968004,2.435107,0.046528,0.956390,0.953226,0.922844,1,1,1
4,25,63,1.000000,3.488957,3.503871,4.076448,0.034465,0.971347,0.971759,0.983872,1,1,1


### Visualize data

In [132]:
def logit_plot(dist, logit, RI):
    canvas = toyplot.Canvas(width=500, height=250)
    ax0 = canvas.cartesian(
        label="pooled data (function)",
        xlabel="Genetic dist.",
        ylabel="Logit function",
        grid=(1, 2, 0),
    )
    ax1 = canvas.cartesian(
        label="pooled data (observation)",
        xlabel="Genetic dist.",
        ylabel="RI",
        grid=(1, 2, 1),
    )

    # points are jittered on x-axis for visibility
    ax0.scatterplot(
        dist,
        logit,
        size=5,
        opacity=0.33,
        color=toyplot.color.Palette()[0],
    );
    ax1.scatterplot(
        dist,
        RI,
        size=10,
        opacity=0.2,
        marker="|",
        mstyle={
            "stroke": toyplot.color.Palette()[1],
            "stroke-width": 3,
        },
    );
    return canvas, (ax0, ax1)

In [133]:
logit_plot(SAMPLE.dist, SAMPLE.logit_b, SAMPLE.RI_pooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t29ef558c30d1433fbfc60c2ef5b6798b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:rgb(40%,76.1%,64.7%);stroke-opacit

In [134]:
logit_plot(SAMPLE.dist, SAMPLE.logit, SAMPLE.RI_unpooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t4780cae49b8e481e80bdb7682f41f6e2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:rgb(40%,76.1%,64

In [135]:
logit_plot(SAMPLE.dist, SAMPLE.logit_x, SAMPLE.RI_partpooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t80214de81d9e496498c7f2b7fb72464e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);

### Define models

In [137]:
def pooled_logistic(x, y, **kwargs):
    
    # define model
    with pm.Model() as model:  

        # parameters and error
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        
        # link function
        effect = 𝛼 + 𝛽 * x
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood
        y = pm.Bernoulli("y", p=logit, observed=y)
        
        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [138]:
def unpooled_logistic(x, y, idx0, idx1, **kwargs):
    
    # define model
    with pm.Model() as model:
        
        # indexers
        sidx0 = pm.Data("spp_idx0", idx0.values)
        sidx1 = pm.Data("spp_idx1", idx1.values)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=1)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=1)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean + 𝜓_std * 𝜓_offset)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        
        # link function
        effect = 𝛼 + (𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * x
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood
        y = pm.Bernoulli("y", p=logit, observed=y)
        
        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [139]:
def partpooled_logistic(x, y, idx0, idx1, gidx, **kwargs):
    
    # define model
    with pm.Model() as model:
        
        # indexers
        sidx0 = pm.Data("spp_idx0", idx0)
        sidx1 = pm.Data("spp_idx1", idx1)
        gidx = pm.Data("gidx", gidx)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=4)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=4)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean[gidx] + 𝜓_std[gidx] * 𝜓_offset)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        
        # linear model prediction
        effect = 𝛼 + (𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * x
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood (normal distributed errors)
        y = pm.Bernoulli("y", p=logit, observed=y)

        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]

        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

### Functions to plot results

In [ ]:
def toytrace(trace, var_names, titles):
    """
    Plot posterior trace with toyplot
    """
    nvars = len(var_names)
    
    # setup canvase
    canvas = toyplot.Canvas(width=500, height=200 * nvars)
    
    # store axes
    axes = []
    
    # iter over params
    for pidx, param in enumerate(var_names):
        
        # get param posterior
        posterior = trace.get_values(param)
        
        # setup axes 
        ax = canvas.cartesian(grid=(nvars, 1, pidx))
        ax.y.show = False
        ax.x.spine.style = {"stroke-width": 1.5}
        ax.x.ticks.labels.style = {"font-size": "12px"}
        ax.x.ticks.show = True
        ax.x.label.text = f"param='{titles[pidx]}'"        
        
        # iterate over shape of param
        for idx in range(posterior.shape[1]):
            mags, bins = np.histogram(posterior[:, idx], bins=100)
            ax.plot(bins[1:], mags, stroke_width=2, opacity=0.6)
        axes.append(ax)
    return canvas, axes

In [152]:
import scipy.stats as stats

def draw_velocity_dists(trace, baseline=0.15):
    """
    Draw the clade velocities as gaussians
    """
    canvas = toyplot.Canvas(width=350, height=300)
    axes = canvas.cartesian(xlabel="Relative velocity of reproductive isolation")
    marks = []
    base = 0
    for i in range(trace['𝜓_mean'].shape[1]):
        
        loc = trace['𝜓_mean'][:, i].mean()
        scale = trace['𝜓_std'][:, i].mean()
        interval = stats.norm.interval(0.995, loc, scale)
        points = np.linspace(interval[0], interval[1], 100)
        mark = axes.fill(
            points, 
            stats.norm.pdf(points, loc=loc, scale=scale), 
            style={
                "fill-opacity": 0.45,
                "stroke": 'black',
                "stroke-opacity": 1.0,
                "stroke-width": 1,
            },
            baseline=np.repeat(base, 100),
        )
        marks.append(mark)
        axes.hlines(base, style={"stroke-dasharray": "5,5", 'stroke-width': 1})
        base += baseline
        
    axes.y.show = False
    axes.x.ticks.locator = toyplot.locator.Extended(only_inside=True)
    axes.x.ticks.show = True
    return canvas, axes, marks

In [252]:
# MCMC sampler kwargs
sample_kwargs = dict(
    tune=10000,
    draws=10000,
    target_accept=0.99,
    return_inferencedata=False,
    progressbar=True,
)

### Run three datasets under pooled model

In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_pooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_unpooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_partpooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

### Run three datasets under unpooled model

In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_pooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_unpooled_data  = unpooled_logistic(*model_args[:4], **sample_kwargs)

In [262]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_partpooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 7965 seconds.


### Run three datasets under partpooled model

In [264]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_pooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 4047 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [265]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_unpooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 3821 seconds.


In [266]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_partpooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 3994 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [269]:
partpooled_model_partpooled_data['stats']

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
𝜓_mean[0],1.580,2.259,-2.709,5.750,0.025,0.018,8141.0,8141.0,8141.0,13902.0,1.0
𝜓_mean[1],1.223,2.289,-3.024,5.567,0.025,0.018,8275.0,8275.0,8275.0,14835.0,1.0
𝜓_mean[2],-0.332,2.257,-4.526,3.966,0.025,0.018,8117.0,8117.0,8115.0,13956.0,1.0
𝜓_mean[3],-1.304,2.261,-5.450,3.071,0.025,0.018,8081.0,8081.0,8079.0,13768.0,1.0
𝜓_offset[0],-0.179,0.924,-1.869,1.623,0.004,0.005,46267.0,17093.0,46545.0,26862.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
logit[995],0.977,0.016,0.950,0.999,0.000,0.000,40297.0,40297.0,45532.0,32304.0,1.0
logit[996],0.977,0.016,0.951,0.999,0.000,0.000,37560.0,37560.0,37165.0,30249.0,1.0
logit[997],0.981,0.013,0.959,1.000,0.000,0.000,31533.0,31477.0,29917.0,29832.0,1.0
logit[998],0.942,0.048,0.855,0.996,0.000,0.000,33769.0,33532.0,31239.0,30602.0,1.0


In [270]:
toytrace(partpooled_model_partpooled_data['trace'], ['𝜓_mean', '𝜓_offset', '𝜓'], ['psi-mean', 'psi-offset', 'psi-spp']);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t61eb00d46a5c41e8a01dbc1718cfb1e6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 600.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 111.35957059564538 149.9203821656051 L 114.78018099366916 150.0 L 118.20079139169295 150.0 L 121.62140178971673 150.0 L 125.04201218774051 150.0 L 128.46262258576428 149.84076433121018 L 131.88323298378808 149.84076433121018 L 135.30384338181187 149.60191082802547 L 138.72445377983564 149.9203821656051 L 142.14506417785944 149.7611464968153 L 145.56567457588324 149.7611464968153 L 148.986284973907 149.28343949044586 L 152.40689537193077 149.52229299363057 L 155.8275057699546 149.12420382165604 L 159.24811616797837 148.72611464968153 L 162.66872656600216 148.72611464968153 L 166.08933696402593 147.92993630573247 L 169.50994736204973 148.2484076433121 L 172.9305577600735 147.8503184713376 L 176.35116815809727 146.7356687898089 L 179.77177855612106 146.5764331210191 L 183.19238895414486 145.30254777070064 L 186.61299935216866 144.5859872611465 L 190.0336097501924 142.9936305732484 L 193.45422014821622 141.40127388535032 L 196.87483054623996 139.72929936305732 L 200.2954409442638 137.89808917197453 L 203.71605134228756 136.70382165605096 L 207.1366617403113 132.64331210191082 L 210.55727213833512 130.57324840764332 L 213.97788253635892 127.54777070063695 L 217.39849293438272 126.19426751592356 L 220.81910333240646 121.01910828025477 L 224.23971373043025 114.57006369426753 L 227.66032412845405 114.49044585987262 L 231.08093452647782 111.14649681528661 L 234.5015449245016 107.16560509554141 L 237.92215532252538 99.36305732484075 L 241.34276572054918 96.17834394904457 L 244.76337611857295 92.19745222929936 L 248.18398651659675 89.88853503184714 L 251.60459691462052 86.14649681528661 L 255.02520731264428 78.26433121019107 L 258.4458177106681 77.30891719745223 L 261.8664281086919 71.73566878980893 L 265.28703850671565 68.47133757961784 L 268.7076489047395 62.10191082802547 L 272.1282593027633 61.62420382165605 L 275.548869700787 57.1656050955414 L 278.9694800988108 55.49363057324841 L 282.3900904968346 56.68789808917198 L 285.8107008948584 57.56369426751592 L 289.2313112928821 52.22929936305732 L 292.6519216909059 58.99681528662421 L 296.07253208892973 61.46496815286624 L 299.4931424869535 65.44585987261146 L 302.9137528849772 68.23248407643311 L 306.3343632830011 65.92356687898089 L 309.75497368102486 71.3375796178344 L 313.17558407904863 77.38853503184714 L 316.5961944770724 77.30891719745223 L 320.0168048750962 82.24522292993629 L 323.43741527312 87.26114649681529 L 326.85802567114376 89.01273885350318 L 330.2786360691676 95.85987261146497 L 333.69924646719136 104.37898089171975 L 337.1198568652151 109.31528662420382 L 340.5404672632389 110.50955414012738 L 343.9610776612627 114.72929936305732 L 347.3816880592865 120.22292993630575 L 350.80229845731026 119.10828025477706 L 354.222908855334 123.48726114649682 L 357.64351925335785 126.671974522293 L 361.06412965138156 130.57324840764332 L 364.4847400494054 134.87261146496814 L 367.9053504474292 136.38535031847132 L 371.325960845453 138.77388535031847 L 374.7465712434767 140.7643312101911 L 378.1671816415005 142.6751592356688 L 381.58779203952435 142.35668789808918 L 385.0084024375481 144.5063694267516 L 388.4290128355718 143.86942675159236 L 391.84962323359565 146.5764331210191 L 395.2702336316195 147.531847133758 L 398.69084402964324 147.77070063694268 L 402.11145442766696 147.61146496815286 L 405.53206482569084 148.16878980891718 L 408.9526752237146 148.6464968152866 L 412.3732856217383 148.88535031847132 L 415.7938960197621 149.283439

In [271]:
# show plot of TRUE vs. ESTIMATED rates
c, a, m = toyplot.scatterplot(
    partpooled_model_partpooled_data['trace']['𝜓'].mean(axis=0),         # estimated
    SPECIES_DATA['psi_x'],                             # true
    width=400,
    height=250,
    xlabel="ESTIMATED species velocity",
    ylabel="TRUE species velocity",
    color=[toyplot.color.Palette()[i] for i in SPECIES_DATA.gidx],
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t8d54a9ecd8874a34b4ee3d33449fe002" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 250.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> -2 -1 0 1 2 ESTIMATED species velocity -1 0 1 TRUE species velocity

In [274]:
draw_velocity_dists(partpooled_model_partpooled_data['trace'], baseline = 1.8);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="te15368dc83fe47e1a529b0022df12573" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="160.4549109685148,250.0 161.84657661391236,250.0 163.23824225930997,250.0 164.62990790470758,250.0 166.02157355010522,250.0 167.41323919550283,250.0 168.80490484090043,250.0 170.19657048629804,250.0 171.58823613169568,250.0 172.97990177709326,250.0 174.37156742249087,250.0 175.76323306788848,250.0 177.15489871328606,250.0 178.5465643586837,250.0 179.93823000408128,250.0 181.32989564947889,250.0 182.7215612948765,250.0 184.1132269402741,250.0 185.50489258567174,250.0 186.89655823106932,250.0 188.28822387646693,250.0 189.67988952186454,250.0 191.07155516726215,250.0 192.46322081265978,250.0 193.8548864580574,250.0 195.24655210345497,250.0 196.6382177488526,250.0 198.0298833942502,250.0 199.4215490396478,250.0 200.8132146850454,250.0 202.20488033044302,250.0 203.59654597584063,250.0 204.98821162123824,250.0 206.37987726663587,250.0 207.77154291203345,250.0 209.16320855743103,250.0 210.55487420282867,250.0 211.94653984822628,250.0 213.3382054936239,250.0 214.7298711390215,250.0 216.12153678441908,250.0 217.51320242981672,250.0 218.9048680752143,250.0 220.2965337206119,250.0 221.6881993660095,250.0 223.07986501140712,250.0 224.47153065680476,250.0 225.8631963022024,250.0 227.25486194759995,250.0 228.64652759299753,250.0 230.03819323839517,250.0 231.42985888379278,250.0 232.8215245291904,250.0 234.213190174588,250.0 235.60485581998563,250.0 236.99652146538318,250.0 238.3881871107808,250.0 239.77985275617843,250.0 241.17151840157607,250.0 242.56318404697365,250.0 243.95484969237123,250.0 245.3465153377689,250.0 246.73818098316644,250.0 248.12984662856408,250.0 249.52151227396166,250.0 250.9131779193593,250.0 252.3048435647569,250.0 253.69650921015452,250.0 255.0881748555521,250.0 256.4798405009497,250.0 257.8715061463473,250.0 259.2631717917449,250.0 260.65483743714253,250.0 262.04650308254014,250.0 263.43816872793775,250.0 264.82983437333536,250.0 266.22150001873297,250.0 267.6131656641306,250.0 269.00483130952824,250.0 270.39649695492574,250.0 271.7881626003234,250.0 273.17982824572096,250.0 274.5714938911186,250.0 275.96315953651623,250.0 277.3548251819138,250.0 278.74649082731145,250.0 280.13815647270906,250.0 281.52982211810667,250.0 282.9214877635043,250.0 284.3131534089019,250.0 285.7048190542995,250.0 287.0964846996971,250.0 288.48815034509465,250.0 289.8798159904924,250.0 291.27148163588987,250.0 292.66314728128754,250.0 294.0548129266851,250.0 295.4464785720827,250.0 296.83814421748036,250.0 298.22980986287797,250.0 298.22980986287797,249.56392574868534 296.83814421748036,249.48950276429315 295.4464785720827,249.4042970444097 294.0548129266851,249.30710165482353 292.66314728128754,249.19663537821043 291.27148163588987,249.0715483547577 289.8798159904924,248.93042981812613 288.48815034509465,248.7718180999361 287.0964846996971,248.59421304843417 285.7048190542995,248.39609097117724 284.3131534089019,248.1759221673003 282.9214877635043,247.9321910623105 281.52982211810667,247.66341889778084 280.13815647270906,247.3681888605342 278.74649082731145,247.04517346199532 277.3548251819138,246.69316389978755 275.96315953651623,246.31110105215345 274.5714938911186,245.8981076735086 273.17982824572096,245.45352127881262 271.7881626003234,244.9769271281181 270.39649695492574,244.46819065346426 269.00483130952824,243.92748861113142 267.6131656641306,243.3553381960593 266.22150001873297,242.75262332473963 264.82983437333536,242.1206172806528 263.43816872793775,241.46100092451647 262.04

### Assess model fit

In [165]:
def rmse(predictions, targets):
    differences = predictions - targets                       #the DIFFERENCEs.
    differences_squared = differences ** 2                    #the SQUAREs of ^
    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^
    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^
    return rmse_val   

In [166]:
def aicm(mean, variance):
    return 2*mean - 2*variance

In [167]:
rmse(unpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi'])

0.9384409617057597

In [168]:
rmse(partpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi_x'])

0.6105422350829149

In [177]:
aicm(unpooled_sub['trace']['𝜓'].mean(), unpooled_sub['trace']['𝜓'].var())

-24.454868204001738

In [179]:
# Raftery et al. (2007) holds that this value is better.  R+M disagree?
aicm(partpooled_sub['trace']['𝜓'].mean(), partpooled_sub['trace']['𝜓'].var())

-12.49733802346165

In [275]:
az_pooled_model_pooled_data = az.from_pymc3(trace = pooled_model_pooled_data['trace'], 
                                            model = pooled_model_pooled_data['model'])
az_pooled_model_unpooled_data = az.from_pymc3(trace = pooled_model_unpooled_data['trace'], 
                                            model = pooled_model_unpooled_data['model'])
az_pooled_model_partpooled_data = az.from_pymc3(trace = pooled_model_partpooled_data['trace'], 
                                            model = pooled_model_partpooled_data['model'])
az_unpooled_model_pooled_data = az.from_pymc3(trace = unpooled_model_pooled_data['trace'], 
                                            model = unpooled_model_pooled_data['model'])
az_unpooled_model_unpooled_data = az.from_pymc3(trace = unpooled_model_unpooled_data['trace'], 
                                            model = unpooled_model_unpooled_data['model'])
az_unpooled_model_partpooled_data = az.from_pymc3(trace = unpooled_model_partpooled_data['trace'], 
                                            model = unpooled_model_partpooled_data['model'])
az_partpooled_model_pooled_data = az.from_pymc3(trace = partpooled_model_pooled_data['trace'], 
                                            model = partpooled_model_pooled_data['model'])
az_partpooled_model_unpooled_data = az.from_pymc3(trace = partpooled_model_unpooled_data['trace'], 
                                            model = partpooled_model_unpooled_data['model'])
az_partpooled_model_partpooled_data = az.from_pymc3(trace = partpooled_model_partpooled_data['trace'], 
                                            model = partpooled_model_partpooled_data['model'])

In [277]:
az.compare({"pooled_model_pooled_data": az_pooled_model_pooled_data,
            "pooled_model_unpooled_data": az_pooled_model_unpooled_data, 
            "pooled_model_partpooled_data": az_pooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
pooled_model_unpooled_data,0,-285.25,2.02773,0,0.50183,19.9498,0,False,log
pooled_model_pooled_data,1,-287.587,2.06752,2.33685,0.493347,19.2441,26.7242,False,log
pooled_model_partpooled_data,2,-331.518,1.88475,46.268,0.00482277,18.6164,25.4063,False,log


In [278]:
az.compare({"unpooled_model_pooled_data": az_unpooled_model_pooled_data,
            "unpooled_model_unpooled_data": az_unpooled_model_unpooled_data, 
            "unpooled_model_partpooled_data": az_unpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
unpooled_model_pooled_data,0,-288.316,9.745,0,0.855715,20.0511,0,False,log
unpooled_model_partpooled_data,1,-318.154,35.5724,29.8374,0.0769441,19.9981,27.061,False,log
unpooled_model_unpooled_data,2,-318.293,35.6964,29.9772,0.0673405,19.9866,27.0695,False,log


In [279]:
az.compare({"partpooled_model_pooled_data": az_partpooled_model_pooled_data,
            "partpooled_model_unpooled_data": az_partpooled_model_unpooled_data, 
            "partpooled_model_partpooled_data": az_partpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
partpooled_model_unpooled_data,0,-289.825,18.9114,0,0.402004,21.048,0,False,log
partpooled_model_pooled_data,1,-291.637,21.5293,1.81223,0.380538,19.6133,27.8522,False,log
partpooled_model_partpooled_data,2,-297.684,19.1278,7.8595,0.217458,20.0491,25.7225,False,log


In [280]:
az.compare({"pooled_model_pooled_data": az_pooled_model_pooled_data,
            "unpooled_model_unpooled_data": az_unpooled_model_unpooled_data, 
            "partpooled_model_partpooled_data": az_partpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
pooled_model_pooled_data,0,-287.587,2.06752,0,0.637319,19.9557,0,False,log
partpooled_model_partpooled_data,1,-297.684,19.1278,10.0973,0.362638,19.6385,27.043,False,log
unpooled_model_unpooled_data,2,-318.293,35.6964,30.7062,4.27749e-05,19.1133,26.9696,False,log
